<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/draft/football_live_EDA_heft_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project config

In [1]:
try:
    import neptune.new as neptune
except:
    !pip install neptune-client >> None
    import neptune.new as neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('credential.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw

/usr/local/lib/python3.10/dist-packages/neptune/internal/backends/hosted_client.py:50: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version
<ipython-input-1-94b4efcbc4c0>:5: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  import neptune.new as neptune


In [2]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

## Main Data

### Downloads

In [ ]:
!wget -q -O ./data.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/t3GXEat3tuZ36w

In [ ]:
!unrar l ./data.rar

In [ ]:
!mkdir -p ./Results
!unrar e ./data.rar data.csv ./Results/

In [ ]:
!mkdir -p ./Prem
!unrar e ./data.rar prem.csv ./Prem/

In [ ]:
!mkdir -p ./Index
!unrar e ./data.rar info.csv  ./Index/

In [ ]:
!unrar e ./data.rar holdout_id.csv ./

In [ ]:
!wc -l ./Results/data.csv
#14899849 ./data.csv

### Imports

In [5]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

1.5.3
1.22.4


In [6]:
np.iinfo(np.int8).max, np.iinfo(np.int16).max, np.iinfo(np.int32).max

(127, 32767, 2147483647)

In [7]:
np.finfo(np.float16).precision, np.finfo(np.float32).precision, np.finfo(np.float64).precision

(3, 6, 15)

In [8]:
import os, psutil
from neptune.utils import stringify_unsupported
from tqdm import tqdm
import gc

### Code

#####Functions

In [9]:
# Присоединяем итоговый результат и по первому тайму
def add_match_results (data_df:pd.DataFrame, cols = ['Id', 'Result1', 'Result2', 'Periods'], info_path = '/content/Index/info.csv'):
    info_df = pd.read_csv(info_path, sep = ';', usecols = cols)
    info_df[['Period1', 'Period2', 'Period3', 'Period4']] = info_df['Periods'].str.split(',', expand = True)
    info_df[['Time1Res1', 'Time1Res2']] = info_df['Period1'].str.split(':', expand = True)
    info_df = info_df[~info_df['Id'].duplicated(keep = False)]
    period1_result_dict = info_df.set_index('Id')[['Time1Res1', 'Time1Res2', 'Result1', 'Result2']].to_dict(orient = 'index')
    #data_df.loc[:, ['Time1Res1', 'Time1Res2', 'Result1', 'Result2']] =  \
    return [[
        #period1_result_dict[id]['Time1Res1'], period1_result_dict[id]['Time1Res1'], 
        period1_result_dict[id]['Result1'], period1_result_dict[id]['Result2']] 
    if id in period1_result_dict else [None, None] for id in tqdm(data_df['Id'].values, total = len(data_df))]
    

In [10]:
# Добавляем прематчевые линии
def add_match_lines (data_df:pd.DataFrame, cols = ['P1', 'P2'], prem_path = '/content/Prem/prem.csv'):
    prem_df = pd.read_csv(prem_path, sep = ';')
    prem_df = prem_df[~prem_df['Id'].duplicated(keep = False)]
    prem_dict = prem_df.set_index('Id')[cols].to_dict(orient = 'index')
    #data_df.loc[:,['P1', 'PX', 'P2', 'PR']] =  \
    return [[prem_dict[id]['P1'], prem_dict[id]['P2']] 
    if id in prem_dict else [None, None] for id in tqdm(data_df['Id'].values, total = len(data_df))]

In [11]:
def transform_dataset(data_df:pd.DataFrame, remain = [], relative = True):
    new_match_vector = data_df['Id'] != data_df['Id'].shift(1)
    # трансформируем минуты
    data_df['min_norm'] = data_df['Minute'].astype(np.float32) / 50
    print('1. Минуты посчитаны...')
    # трансформируем голы
    data_df[data_df['Score1'].isna() & new_match_vector] = 0
    data_df['Score1_norm'] = data_df['Score1'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score1'] > 3, ['Score1_norm']] = 1.0
    data_df[data_df['Score2'].isna() & new_match_vector] = 0
    data_df['Score2_norm'] = data_df['Score2'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score2'] > 3, ['Score2_norm']] = 1.0

    data_df['Score_diff'] = data_df['Score1'].astype(np.int16) - data_df['Score2'].astype(np.int16)
    data_df.loc[data_df['Score_diff'] < -4, ['Score_diff']] = -4
    data_df.loc[data_df['Score_diff'] > 4, ['Score_diff']] = 4
    data_df[[f'Score_cat_{n}' for n in range(1, 10)]] = pd.get_dummies(data_df['Score_diff']).values
    data_df['Score_diff'] = data_df['Score_diff'].astype(np.float32) / np.float32(4.0)
    if 'Score1' not in remain:
        data_df = data_df.drop(['Score1', 'Score2'], axis = 1)
    print('2. Голы посчитаны...')
    #трансформируем атаки
    data_df['A1_scaled'] = data_df['A1'].astype(np.float32) / 75
    data_df.loc[data_df['A1'] >= 60, ['A1_scaled']] = (60 + (data_df['A1'] - 60) / 4) / 75
    data_df['A2_scaled'] = data_df['A2'].astype(np.float32) / 75
    data_df.loc[data_df['A2'] >= 60, ['A2_scaled']] = (60 + (data_df['A2'] - 60) / 4) / 75
    # атаки в минуту
    data_df['A1perMIN'] = data_df['A1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A1perMIN'] > 4, ['A1perMIN']] = np.float32(4.0)
    data_df['A2perMIN'] = data_df['A2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A2perMIN'] > 4, ['A2perMIN']] = np.float32(4.0)
    # динамика атак
    if relative:
        new_match_vector5 = data_df['Id'] != data_df['Id'].shift(5)
        data_df['A1relativ'] = data_df['A1'].astype(np.float32) - data_df['A1'].shift(5).astype(np.float32)
        data_df.loc[new_match_vector5, ['A1relativ']] = np.float32(0.0)
        data_df['A1relativ'] = data_df['A1relativ'].fillna(0)
        data_df.loc[data_df['A1relativ'] > 15, ['A1relativ']] = np.float32(15.)
        data_df['A2relativ'] = data_df['A2'].astype(np.float32) - data_df['A2'].shift(5).astype(np.float32)
        data_df.loc[new_match_vector5, ['A2relativ']] =  np.float32(0.0)
        data_df['A2relativ'] = data_df['A2relativ'].fillna(0)
        data_df.loc[data_df['A2relativ'] > 15, ['A2relativ']] = np.float32(15.)
    if 'A1' not in remain:
        data_df = data_df.drop(['A1', 'A2'], axis = 1)
    print('3. Атаки посчитаны...')
    # трансформируем опасные атаки
    data_df['DA1_scaled'] = data_df['DA1'].astype(np.float32) / 50
    data_df.loc[data_df['DA1'] >= 40, ['DA1_scaled']] = (80 + (data_df['DA1'] - 40) / 3) / 100
    data_df['DA2_scaled'] = data_df['DA2'].astype(np.float32) / 50
    data_df.loc[data_df['DA2'] >= 40, ['DA2_scaled']] = (80 + (data_df['DA2'] - 40) / 3) / 100
    # опасные атаки в минуту    
    data_df['DA1perMIN'] = data_df['DA1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA1perMIN'] > 3, ['DA1perMIN']] = np.float32(3.0)
    data_df['DA2perMIN'] = data_df['DA2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA2perMIN'] > 3, ['DA2perMIN']] = np.float32(3.0)
    # динамика опасных атак
    if relative:
        data_df['DA1relativ'] = data_df['DA1'].astype(np.float32) - data_df['DA1'].shift(5).astype(np.float32)
        data_df.loc[new_match_vector5, ['DA1relativ']] = np.float32(0.0)
        data_df['DA1relativ'] = data_df['DA1relativ'].fillna(0)
        data_df.loc[data_df['DA1relativ'] > 10, ['DA1relativ']] = np.float32(10.)
        data_df['DA2relativ'] = data_df['DA2'].astype(np.float32) - data_df['DA2'].shift(5).astype(np.float32)
        data_df.loc[new_match_vector5, ['DA2relativ']] = np.float32(0.0)
        data_df['DA2relativ'] = data_df['DA2relativ'].fillna(0)
        data_df.loc[data_df['DA2relativ'] > 10, ['DA2relativ']] = np.float32(10.)
    if 'DA1' not in remain:
        data_df = data_df.drop(['DA1', 'DA2'], axis = 1)
    if 'Minute' not in remain:
        data_df = data_df.drop(['Minute'], axis = 1)
    print('4. Опасные атаки посчитаны...')
    # трансформируем владение мячом
    data_df[data_df['Pos1'].isna() & new_match_vector] = 0
    data_df['Pos1_cleaned'] = data_df['Pos1'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos1_cleaned'] < 0.2, ['Pos1_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos1_cleaned'] > 0.8, ['Pos1_cleaned']] = np.float32(0.8)
    data_df[data_df['Pos2'].isna() & new_match_vector] = 0
    data_df['Pos2_cleaned'] = data_df['Pos2'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos2_cleaned'] < 0.2, ['Pos2_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos2_cleaned'] > 0.8, ['Pos2_cleaned']] = np.float32(0.8)
    if 'Pos1' not in remain:
        data_df = data_df.drop(['Pos1', 'Pos2'], axis = 1)
    print('5. Владение мячом посчитпно...')
    # трансформируем удары
    data_df[data_df['Off1'].isna() & new_match_vector] = 0
    data_df['Off1_norm'] = data_df['Off1'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off1_norm'] > 1.0, ['Off1_norm']] = np.float32(1.0)
    data_df[data_df['Off2'].isna() & new_match_vector] = 0
    data_df['Off2_norm'] = data_df['Off2'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off2_norm'] > 1.0, ['Off2_norm']] = np.float32(1.0)
    if 'Off1' not in remain:
        data_df = data_df.drop(['Off1', 'Off2'], axis = 1)
    print('6. Удары посчитаны...')
    # трансформируем удары в створ
    data_df[data_df['On1'].isna() & new_match_vector] = 0
    data_df['On1_norm'] = data_df['On1'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On1_norm'] > 1.0, ['On1_norm']] = np.float32(1.0)
    data_df[data_df['On2'].isna() & new_match_vector] = 0
    data_df['On2_norm'] = data_df['On2'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On2_norm'] > 1.0, ['On2_norm']] = np.float32(1.0)
    if 'On1' not in remain:
        data_df = data_df.drop(['On1', 'On2'], axis = 1)    
    print('7. Удары в створ посчитаны...')
    # трансформируем желтые карточки
    data_df[data_df['YC1'].isna() & new_match_vector] = 0
    data_df['YC1_transformed'] = data_df['YC1'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC1_transformed'] > 1.0, ['YC1_transformed']] = np.float32(1.0)
    data_df[data_df['YC2'].isna() & new_match_vector] = 0
    data_df['YC2_transformed'] = data_df['YC2'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC2_transformed'] > 1.0, ['YC2_transformed']] = np.float32(1.0)
    if 'YC1' not in remain:
        data_df = data_df.drop(['YC1', 'YC2'], axis = 1)
    print('8. Жёлтые карточки посчитаны...')
    # трансформируем красные карточки
    data_df[data_df['RC1'].isna() & new_match_vector] = 0
    data_df['RC1_transformed'] = data_df['RC1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC1_transformed'] > 1, ['RC1_transformed']] = np.int8(1)
    data_df[data_df['RC2'].isna() & new_match_vector] = 0
    data_df['RC2_transformed'] = data_df['RC2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC2_transformed'] > 1, ['RC2_transformed']] = np.int8(1)
    if 'RC1' not in remain:
        data_df = data_df.drop(['RC1', 'RC2'], axis = 1)
    print('9. Красные карточки посчитаны...')
    # трансформируем замены
    data_df[data_df['Sub1'].isna() & new_match_vector] = 0
    data_df['Sub1_transformed'] = data_df['Sub1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub1_transformed'] > 1, ['Sub1_transformed']] = np.int8(1)
    data_df[data_df['Sub2'].isna() & new_match_vector] = 0
    data_df['Sub2_transformed'] = data_df['Sub2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub2_transformed'] > 1, ['Sub2_transformed']] = np.int8(1)
    if 'Sub1' not in remain:
        data_df = data_df.drop(['Sub1', 'Sub2'], axis = 1)
    print('10. Замены посчитаны...')
    # трансформируем угловык
    data_df[data_df['Cor1'].isna() & new_match_vector] = 0
    data_df['Cor1_transformed'] = data_df['Cor1'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor1_transformed'] > 1.0, ['Cor1_transformed']] = np.float32(1.0)
    data_df[data_df['Cor2'].isna() & new_match_vector] = 0
    data_df['Cor2_transformed'] = data_df['Cor2'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor2_transformed'] > 1.0, ['Cor2_transformed']] = np.float32(1.0)
    if 'Cor1' not in remain:
        data_df = data_df.drop(['Cor1', 'Cor2'], axis = 1)
    print('11. Угловые посчитаны...')
    # трансформируем линию
    data_df['P1_transformed'] = np.log(data_df['P1'], dtype = np.float32) / 2
    data_df['P2_transformed'] = np.log(data_df['P2'], dtype = np.float32) / 2
    if 'P1' not in remain:
        data_df = data_df.drop(['P1', 'P2'], axis = 1)
    print('12. Линии посчитаны...')
    if 'Pen1' not in remain:
        data_df = data_df.drop(['Pen1', 'Pen2'], axis = 1)
    if 'Active' not in remain:
        data_df = data_df.drop(['Active'], axis = 1)
    return data_df

#### Predefine data

In [12]:
data_types_dict = {
    'Id':np.int32, 
    'StatTime':np.datetime64, 
    'Minute':np.int8, 
    'Active': np.int8, 'Score1':np.int8, 'Score2':np.int8,
    'A1':np.int16, 'A2':np.int16, 'DA1':np.int16, 'DA2':np.int16, 'Pos1':np.float32, 'Pos2':np.float32,
    'Off1':np.int8, 'Off2':np.int8, 'On1':np.int8, 'On2':np.int8, 'YC1':np.int8, 'YC2':np.int8,
    'RC1':np.int8, 'RC2':np.int8, 'Sub1':np.int8, 'Sub2':np.int8, 'Pen1':np.int8, 'Pen2':np.int8,
    'Cor1':np.int8, 'Cor2':np.int8, 'Period':np.int8, 
    'D':np.datetime64,
    'I':np.int32, 'Active.1':np.int8, 
    'Time':np.datetime64, 
    'Minute.1':np.int8, 
    'RawTime':np.datetime64,
    'Score1.1':np.int8, 'Score2.1':np.int8, 'Period.1':np.int8, 
    'W1':np.float16, 'WX':np.float16, 'W2':np.float16, 'X1':np.float16, 'X2':np.float16, 'W12':np.float16, 'TotalValue':np.float16, 
    'Over':np.float16, 'Under':np.float16, 'Hand1Value':np.float16, 'H1':np.float16, 'H2':np.float16 
}

In [13]:
drop_cols = [
    #'StatTime', 
    'Comment', 'D', 'I', 'Time', 'Minute.1', 'RawTime',
    'Score1.1', 'Score2.1', 'Period.1', 'Period', 'Periods', 'Serve'
]

In [14]:
k_cols = [
    'W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'TotalValue' ,'Over',
    'Under', 'Hand1Value', 'H1', 'H2' 
]

In [15]:
test_dict = {
    6015559:2, 6053875:3, 6127595:2, 6206288:11, 6368081:3, 6015561:1,
    6054180:3, 6147868:3, 6348541:2, 6015562:2, 6055136:3, 6150224:3, 6354196:2
}
#vector = pd.read_csv('./data.csv', sep = ';', usecols = ['Id'])
#vector_bool = vector['Id'].isin(test_dict)

#### create dataframe

In [16]:
data_df = pd.DataFrame()

In [17]:
for col, col_type in data_types_dict.items(): #{'Id':np.int32 , 'StatTime':np.datetime64}.items(): #
    if col not in drop_cols:
        
        if col == 'Id':
            data_col = pd.read_csv('/content/Results/data.csv', sep = ';', usecols = [col])
            if data_col[col].isna().sum() > 0:
                print(col , ' nan error')
            data_col = data_col.astype(col_type)
            new_match_vector = data_col['Id'] != data_col['Id'].shift(1)
            data_df[col] = data_col
        elif col == 'StatTime':
            data_col = pd.read_csv('/content/Results/data.csv', sep = ';', 
                                   usecols = ['StatTime'], 
                                   parse_dates = ['StatTime'])
            data_col.loc[data_col['StatTime'] == 'NA                 ',['StatTime']] = '31.01.2010 00:00:00'
            #data_col = data_col.astype(col_type)
            data_df[col] = data_col
        elif col in k_cols:
            data_col = pd.read_csv('/content/Results/data.csv', sep = ';', usecols = [col])
            data_col = data_col.fillna(0).astype(col_type)
            data_df[col] = data_col
        else:
            data_col = pd.read_csv('/content/Results/data.csv', sep = ';', usecols = [col])
            data_col[data_col[col].isna() & new_match_vector] = 0
            data_col = data_col.fillna(method = 'ffill').astype(col_type)
            data_df[col] = data_col
        del data_col
        gc.collect()
        #---------------
        process = psutil.Process(os.getpid())
        print(col, ' mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
        #---------------   
    #if col == 'Active':
    #    break

Id  mem usage:  0.45 GiB
StatTime  mem usage:  1.31 GiB
Minute  mem usage:  1.2 GiB
Active  mem usage:  1.32 GiB
Score1  mem usage:  1.34 GiB
Score2  mem usage:  1.22 GiB
A1  mem usage:  1.37 GiB
A2  mem usage:  1.4 GiB
DA1  mem usage:  1.28 GiB
DA2  mem usage:  1.31 GiB
Pos1  mem usage:  1.42 GiB
Pos2  mem usage:  1.48 GiB
Off1  mem usage:  1.59 GiB
Off2  mem usage:  1.6 GiB
On1  mem usage:  1.62 GiB
On2  mem usage:  1.63 GiB
YC1  mem usage:  1.53 GiB
YC2  mem usage:  1.65 GiB
RC1  mem usage:  1.56 GiB
RC2  mem usage:  1.49 GiB
Sub1  mem usage:  1.58 GiB
Sub2  mem usage:  1.52 GiB
Pen1  mem usage:  1.72 GiB
Pen2  mem usage:  1.75 GiB
Cor1  mem usage:  1.75 GiB
Cor2  mem usage:  1.58 GiB
Active.1  mem usage:  1.59 GiB
W1  mem usage:  1.79 GiB
WX  mem usage:  1.83 GiB
W2  mem usage:  1.86 GiB
X1  mem usage:  1.77 GiB
X2  mem usage:  1.8 GiB
W12  mem usage:  1.83 GiB
TotalValue  mem usage:  1.86 GiB
Over  mem usage:  2.0 GiB
Under  mem usage:  1.92 GiB
Hand1Value  mem usage:  2.06 GiB
H1

#### create time.csv

In [18]:
(data_df['Id'].astype(str) + ':' + data_df['StatTime'].astype(str).str.split(
        ' ', expand=True)[0]
        ).drop_duplicates().str.split(':', expand = True).rename(
                                                columns = {0:'Id', 1:'StatTime'}
                                                            ).to_csv('./time.csv', index = False)
data_df = data_df.drop(['StatTime'], axis = 1)

#### add final results

In [19]:
## Добавляем финальный результат
data_df.loc[:, ['Result1', 'Result2']] = add_match_results (data_df)
print('\n Nan: ', data_df['Result1'].isna().sum())
data_df = data_df[~data_df['Result1'].isna()]
data_df[['Result1', 'Result2']] = data_df[['Result1', 'Result2']].astype(np.int8)

## Добавляем линии
data_df.loc[:,['P1', 'P2']] = add_match_lines (data_df)
print('\n Nan: ', data_df['P1'].isna().sum())
data_df = data_df[~data_df['P1'].isna()]
data_df[['P1', 'P2']] = data_df[['P1', 'P2']].astype(np.float16)
#---------------
process = psutil.Process(os.getpid())
print(col, ' mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
#---------------    

100%|██████████| 15228351/15228351 [02:00<00:00, 126512.28it/s]



 Nan:  108


100%|██████████| 15228243/15228243 [02:00<00:00, 126620.63it/s]



 Nan:  107056
H2  mem usage:  2.32 GiB


In [20]:
data_df.tail()

,Id,Minute,Active,Score1,Score2,A1,A2,DA1,DA2,Pos1,Pos2,Off1,Off2,On1,On2,YC1,YC2,RC1,RC2,Sub1,Sub2,Pen1,Pen2,Cor1,Cor2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,P1,P2
15228346,6506727,39,1,1,0,44,33,24,12,0.0,0.0,5,0,1,4,0,2,0,0,0,1,0,0,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1.200195,15.0
15228347,6506727,40,1,1,0,47,33,25,14,0.0,0.0,5,1,1,4,0,2,0,0,0,1,0,0,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1.200195,15.0
15228348,6506727,41,1,1,0,50,33,26,14,0.0,0.0,5,1,1,4,0,2,0,0,0,1,0,0,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1.200195,15.0
15228349,6506727,42,1,1,0,50,34,26,14,0.0,0.0,5,1,1,4,0,2,0,0,0,1,0,0,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1.200195,15.0
15228350,6506727,45,1,1,0,51,34,26,14,0.0,0.0,5,1,1,4,0,2,0,0,0,1,0,0,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1.200195,15.0


#### transform data

In [21]:
data_df = transform_dataset(
    data_df, 
    remain = ['Score1', 'Score2'],
    relative = False
    )

1. Минуты посчитаны...
2. Голы посчитаны...
3. Атаки посчитаны...
4. Опасные атаки посчитаны...
5. Владение мячом посчитпно...
6. Удары посчитаны...
7. Удары в створ посчитаны...
8. Жёлтые карточки посчитаны...
9. Красные карточки посчитаны...
10. Замены посчитаны...
11. Угловые посчитаны...


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


12. Линии посчитаны...


#### delete NaN

In [22]:
print('P1 NaN & P2 NaN: ', data_df['P1_transformed'].isna().sum(), data_df['P2_transformed'].isna().sum())

data_df = data_df[~data_df['P1_transformed'].isna()]
data_df = data_df[~data_df['P2_transformed'].isna()]

print('Score-Result error 1&2: ', (~(data_df['Result1'] - data_df['Score1']) >= 0).sum(), (~(data_df['Result1'] - data_df['Score1']) >= 0).sum())
data_df = data_df.loc[(data_df['Result1'] - data_df['Score1']) >= 0]
data_df = data_df.loc[(data_df['Result2'] - data_df['Score2']) >= 0]


#---------------
process = psutil.Process(os.getpid())
print('mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
#---------------    

P1 NaN & P2 NaN:  0 0
Score-Result error 1&2:  856 856
mem usage:  7.04 GiB


#### some results

In [23]:
data_df.loc[(data_df['Result1'] - data_df['Score1']) < 0]

,Id,Score1,Score2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,min_norm,Score1_norm,Score2_norm,Score_diff,Score_cat_1,Score_cat_2,Score_cat_3,Score_cat_4,Score_cat_5,Score_cat_6,Score_cat_7,Score_cat_8,Score_cat_9,A1_scaled,A2_scaled,A1perMIN,A2perMIN,DA1_scaled,DA2_scaled,DA1perMIN,DA2perMIN,Pos1_cleaned,Pos2_cleaned,Off1_norm,Off2_norm,On1_norm,On2_norm,YC1_transformed,YC2_transformed,RC1_transformed,RC2_transformed,Sub1_transformed,Sub2_transformed,Cor1_transformed,Cor2_transformed,P1_transformed,P2_transformed


In [24]:
((data_df['Result1'] - data_df['Score1']) < 0).sum()

0

In [25]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15119802 entries, 0 to 15228350
Data columns (total 55 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Id                int32  
 1   Score1            int8   
 2   Score2            int8   
 3   Active.1          int8   
 4   W1                float16
 5   WX                float16
 6   W2                float16
 7   X1                float16
 8   X2                float16
 9   W12               float16
 10  TotalValue        float16
 11  Over              float16
 12  Under             float16
 13  Hand1Value        float16
 14  H1                float16
 15  H2                float16
 16  Result1           int8   
 17  Result2           int8   
 18  min_norm          float32
 19  Score1_norm       float32
 20  Score2_norm       float32
 21  Score_diff        float32
 22  Score_cat_1       uint8  
 23  Score_cat_2       uint8  
 24  Score_cat_3       uint8  
 25  Score_cat_4       uint8  
 26  Score_cat_5 

In [26]:
data_df.describe()

,Id,Score1,Score2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,min_norm,Score1_norm,Score2_norm,Score_diff,Score_cat_1,Score_cat_2,Score_cat_3,Score_cat_4,Score_cat_5,Score_cat_6,Score_cat_7,Score_cat_8,Score_cat_9,A1_scaled,A2_scaled,A1perMIN,A2perMIN,DA1_scaled,DA2_scaled,DA1perMIN,DA2perMIN,Pos1_cleaned,Pos2_cleaned,Off1_norm,Off2_norm,On1_norm,On2_norm,YC1_transformed,YC2_transformed,RC1_transformed,RC2_transformed,Sub1_transformed,Sub2_transformed,Cor1_transformed,Cor2_transformed,P1_transformed,P2_transformed
count,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,15119802.0,1.511980e+07,1.511980e+07,15119802.0,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07
mean,3.806749e+06,3.131681e-01,2.491339e-01,3.407605e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.569513e+00,1.271828e+00,4.581242e-01,7.812629e-02,6.218962e-02,1.594242e-02,9.387028e-04,3.834905e-03,2.124241e-02,1.261058e-01,6.506366e-01,1.569092e-01,3.208256e-02,6.453920e-03,1.795923e-03,3.327270e-01,3.182437e-01,1.129196e+00,1.079529e+00,2.749791e-01,2.487535e-01,6.007668e-01,5.416068e-01,4.564157e-01,4.328656e-01,1.426817e-01,1.237993e-01,2.010737e-01,1.705206e-01,1.133528e-01,1.225857e-01,5.708210e-03,6.675881e-03,2.880249e-02,2.953141e-02,1.927124e-01,1.657641e-01,-inf,-inf
std,1.315812e+06,6.144587e-01,5.419506e-01,4.739650e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,1.460551e+00,1.303003e+00,2.596625e-01,1.523816e-01,1.346734e-01,1.982110e-01,3.062387e-02,6.180775e-02,1.441914e-01,3.319685e-01,4.767689e-01,3.637151e-01,1.762194e-01,8.007664e-02,4.234026e-02,2.096327e-01,2.023994e-01,4.463000e-01,4.370320e-01,2.032851e-01,1.904377e-01,3.326157e-01,3.190521e-01,1.649117e-01,1.568069e-01,1.618986e-01,1.477034e-01,2.445237e-01,2.238836e-01,2.450550e-01,2.543316e-01,7.533675e-02,8.143288e-02,1.672510e-01,1.692906e-01,2.275883e-01,2.096369e-01,NaN,NaN
min,3.246400e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,-17.5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-02,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-01,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-inf,-inf
25%,2.704543e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.400000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.600000e-01,1.466667e-01,8.500000e-01,8.064516e-01,1.200000e-01,1.000000e-01,3.793103e-01,3.333333e-01,3.400000e-01,3.100000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.505060e-01,3.940511e-01
50%,3.774122e+06,0.000000e+00,0.000000e+00,0.000000e+00,

In [27]:
data_df.columns

Index(['Id', 'Score1', 'Score2', 'Active.1', 'W1', 'WX', 'W2', 'X1', 'X2',
       'W12', 'TotalValue', 'Over', 'Under', 'Hand1Value', 'H1', 'H2',
       'Result1', 'Result2', 'min_norm', 'Score1_norm', 'Score2_norm',
       'Score_diff', 'Score_cat_1', 'Score_cat_2', 'Score_cat_3',
       'Score_cat_4', 'Score_cat_5', 'Score_cat_6', 'Score_cat_7',
       'Score_cat_8', 'Score_cat_9', 'A1_scaled', 'A2_scaled', 'A1perMIN',
       'A2perMIN', 'DA1_scaled', 'DA2_scaled', 'DA1perMIN', 'DA2perMIN',
       'Pos1_cleaned', 'Pos2_cleaned', 'Off1_norm', 'Off2_norm', 'On1_norm',
       'On2_norm', 'YC1_transformed', 'YC2_transformed', 'RC1_transformed',
       'RC2_transformed', 'Sub1_transformed', 'Sub2_transformed',
       'Cor1_transformed', 'Cor2_transformed', 'P1_transformed',
       'P2_transformed'],
      dtype='object')

In [28]:
data_df

,Id,Score1,Score2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,min_norm,Score1_norm,Score2_norm,Score_diff,Score_cat_1,Score_cat_2,Score_cat_3,Score_cat_4,Score_cat_5,Score_cat_6,Score_cat_7,Score_cat_8,Score_cat_9,A1_scaled,A2_scaled,A1perMIN,A2perMIN,DA1_scaled,DA2_scaled,DA1perMIN,DA2perMIN,Pos1_cleaned,Pos2_cleaned,Off1_norm,Off2_norm,On1_norm,On2_norm,YC1_transformed,YC2_transformed,RC1_transformed,RC2_transformed,Sub1_transformed,Sub2_transformed,Cor1_transformed,Cor2_transformed,P1_transformed,P2_transformed
0,330509,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.02,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.000000,0.066667,0.000000,4.000000,0.00,0.04,0.000000,2.000000,0.2,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.000000,1.125646,0.155305
1,330509,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.04,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.000000,0.066667,0.000000,2.500000,0.00,0.06,0.000000,1.500000,0.2,0.8,0.0,0.1,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.000000,1.125646,0.155305
2,330509,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.06,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.026667,0.133333,0.666667,3.333333,0.00,0.10,0.000000,1.666667,0.2,0.8,0.0,0.1,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.000000,1.125646,0.155305
3,330509,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.08,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.066667,0.146667,1.250000,2.750000,0.02,0.10,0.250000,1.250000,0.2,0.8,0.0,0.1,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.000000,1.125646,0.155305
4,330509,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.10,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.080000,0.200000,1.200000,3.000000,0.04,0.12,0.400000,1.200000,0.2,0.8,0.0,0.1,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.000000,1.125646,0.155305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15228346,6506727,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.78,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.586667,0.440000,1.128205,0.846154,0.48,0.24,0.615385,0.307692,0.2,0.2,0.5,0.0,0.2,0.8,0.0,1.0,0,0,0,1,0.5,0.333333,0.091242,1.354025
15228347,6506727,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.80,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.626667,0.440000,1.175000,0.825000,0.50,0.28,0.625000,0.350000,0.2,0.2,0.5,0.1,0.2,0.8,0.0,1.0,0,0,0,1,0.5,0.333333,0.091242,1.354025
15228348,6506727,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.82,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.666667,0.440000,1.219512,0.804878,0.52,0.28,0.634146,0.341463,0.2,0.2,0.5,0.1,0.2,0.8,0.0,1.0,0,0,0,1,0.5,0.333333,0.091242,1.354025
15228349,6506727,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.84,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.666667,0.453333,1.190476,0.809524,0.52,0.28,0.619048,0.333333,0.2,0.2,0.5,0.1,0.2,0.8,0.0,1.0,0,0,0,1,0.5,0.333333,0.091242,1.354025


#### split data & prepare files for upload

In [ ]:
holdout_id_df = pd.read_csv('./holdout_id.csv', header = None, names = ['id'])
data_vector = ~data_df['Id'].isin(holdout_id_df['id'])

In [ ]:
data_df[data_df['Id'] == 6506727]

In [ ]:
#%%writefile save_discription.txt
np.savez_compressed('./dataset', 
                    X = data_df[data_df.columns[18:]][data_vector].values,
                    y_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[data_vector],
                    y_multi = 1 - np.sign((data_df['Result1'] - data_df['Result2']).values)[data_vector],
                    y_diff = (data_df['Result1'] - data_df['Result2']).values[data_vector],
                    y_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[data_vector],
                    y_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[data_vector],
                    score1 = data_df['Score1'].values[data_vector],
                    score2 = data_df['Score2'].values[data_vector],
                    result1 = data_df['Result1'].values[data_vector],
                    result2 = data_df['Result2'].values[data_vector],
                    K_train = data_df[data_df.columns[3:16]].values[data_vector],
                )

In [ ]:
np.savez_compressed('./additional_data', 
                    id = data_df['Id'][data_vector].values,
                    min = data_df['min_norm'][data_vector].values,
)

In [ ]:
params = {}
params['description'] = 'обновленный датасет основная часть https://disk.yandex.ru/d/t3GXEat3tuZ36w \n' + \
'4 класса задач, бинарная классификация и мультикласс, и под регрессию 2 варианта без разделения на трейн, тест и холдаут'
params['features'] = list(data_df.columns)

#### upload data to neptune.ai

In [ ]:
data_version = 'football_live_main_part_npz_230510/'
project = neptune.init_project(
    project="scomesse/football", 
    api_token = api_key
    )
project[data_version + 'dataset'].upload('./dataset.npz')
project[data_version + 'description'].upload('./save_discription.txt')
project[data_version + 'additional_data'].upload('./additional_data.npz')
project[data_version + 'info'].upload('/content/Index/info.csv')
project[data_version + 'time'].upload('./time.csv')
project[data_version + 'params'] = stringify_unsupported(params)
project.stop()

## Update EDA

### Downloads

In [3]:
!wget -q -O ./data.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/IWvldpeoQcAh5w
!mkdir -p ./Result2
!unrar e ./data.rar ./Result2/

Streaming output truncated to the last 5000 lines.
Extracting  ./Result2/6382527.csv                                         83%  OK 
Extracting  ./Result2/6382528.csv                                         83%  OK 
Extracting  ./Result2/6382529.csv                                         83%  OK 
Extracting  ./Result2/6382530.csv                                         83%  OK 
Extracting  ./Result2/6382531.csv                                         83%  OK 
Extracting  ./Result2/6382532.csv                                         83%  OK 
Extracting  ./Result2/6382533.csv                                         83%  OK 
Extracting  ./Result2/6382534.csv                                         83%  OK 
Extracting  ./Result2/6382535.csv                                         83%  OK 
Extracting  ./Result2/6382536.csv                                         83%  OK 
Extracting  ./Result2/6382565

In [4]:
#!wget -q -O ./Result2.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/MwzjeVAGOZwJaw
!wget -q -O ./Prem.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/oTq97titK56Y2A
!mkdir -p ./Prem2
!unrar e ./Prem.rar ./Prem2/


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from ./Prem.rar

Extracting  ./Prem2/20230417.csv                                           0%  OK 
Extracting  ./Prem2/20230416.csv                                           2%  OK 
Extracting  ./Prem2/20230415.csv                                           5%  OK 
Extracting  ./Prem2/20230414.csv                                           6%  OK 
Extracting  ./Prem2/20230413.csv                                           6%  OK 
Extracting  ./Prem2/20230412.csv                                           7%  OK 
Extracting  ./Prem2/20230411.csv                                           8%  OK 
Extracting  ./Prem2/20230410.csv                                           9%  OK 
Extracting  ./Prem2/20230409.csv                                          10%  OK 
Extracting  ./Prem2/20230408.csv                                    

In [5]:
!wget -q -O ./Index.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/rVBc3eq2SG8Ppw
!mkdir -p ./Index2
!unrar e ./Index.rar ./Index2/


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from ./Index.rar

Extracting  ./Index2/20230110.csv                                          0%  OK 
Extracting  ./Index2/20230109.csv                                          0%  OK 
Extracting  ./Index2/20230108.csv                                          1%  OK 
Extracting  ./Index2/20230107.csv                                          2%  OK 
Extracting  ./Index2/20230106.csv                                          3%  OK 
Extracting  ./Index2/20230105.csv                                          3%  OK 
Extracting  ./Index2/20230104.csv                                          3%  OK 
Extracting  ./Index2/20230103.csv                                          4%  OK 
Extracting  ./Index2/20230102.csv                                          4%  OK 
Extracting  ./Index2/20230101.csv                                  

### Imports

In [6]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

1.5.3
1.22.4


In [7]:
np.iinfo(np.int8).max, np.iinfo(np.int16).max, np.iinfo(np.int32).max

(127, 32767, 2147483647)

In [8]:
np.finfo(np.float16).precision, np.finfo(np.float32).precision, np.finfo(np.float64).precision

(3, 6, 15)

In [9]:
import os, psutil, time
from glob import glob
from neptune.utils import stringify_unsupported
from tqdm import tqdm
import functools
import gc

### Code

#####Functions

In [10]:
def define_files_list(data_start:str, folder:str, date_end = '') -> list:
    '''
    data_start, data_end
    folder: ./folder/
    '''
    if date_end == '':
        return sorted(
            [file 
            for file in glob(folder + '*') 
            if int(file.split('/')[-1].split('.csv')[0]) >= int(date_start)]
            )
    else:
        return sorted(
            [file 
            for file in glob(folder + '*') 
            if int(file.split('/')[-1].split('.csv')[0]) >= int(date_start) & \
                int(file.split('/')[-1].split('.csv')[0]) <= int(date_end)]
            )

In [11]:
def create_id_dict(date_start, folder, cols, date_end = '') -> dict:
    '''
    '''
    return pd.concat(map(
        functools.partial(
            pd.read_csv, sep=';', usecols =  cols
                            ),
                    define_files_list(date_start, folder, date_end = date_end)
                    ), ignore_index=True).drop_duplicates(subset = ['Id'], keep = 'last').set_index('Id').to_dict(orient = 'index')

In [12]:
def transform_dataset(data_df:pd.DataFrame, remain = [], relative = True):
    new_match_vector = data_df['Id'] != data_df['Id'].shift(1)
    # трансформируем минуты
    data_df['min_norm'] = data_df['Minute'].astype(np.float32) / 50
    print('1. Минуты посчитаны...')
    # трансформируем голы
    data_df[data_df['Score1'].isna() & new_match_vector] = 0
    data_df['Score1_norm'] = data_df['Score1'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score1'] > 3, ['Score1_norm']] = 1.0
    data_df[data_df['Score2'].isna() & new_match_vector] = 0
    data_df['Score2_norm'] = data_df['Score2'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score2'] > 3, ['Score2_norm']] = 1.0

    data_df['Score_diff'] = data_df['Score1'].astype(np.int16) - data_df['Score2'].astype(np.int16)
    data_df.loc[data_df['Score_diff'] < -4, ['Score_diff']] = -4
    data_df.loc[data_df['Score_diff'] > 4, ['Score_diff']] = 4
    data_df[[f'Score_cat_{n}' for n in range(1, 10)]] = pd.get_dummies(data_df['Score_diff']).values
    data_df['Score_diff'] = data_df['Score_diff'].astype(np.float32) / np.float32(4.0)
    if 'Score1' not in remain:
        data_df = data_df.drop(['Score1', 'Score2'], axis = 1)
    print('2. Голы посчитаны...')
    #трансформируем атаки
    data_df['A1_scaled'] = data_df['A1'].astype(np.float32) / 75
    data_df.loc[data_df['A1'] >= 60, ['A1_scaled']] = (60 + (data_df['A1'] - 60) / 4) / 75
    data_df['A2_scaled'] = data_df['A2'].astype(np.float32) / 75
    data_df.loc[data_df['A2'] >= 60, ['A2_scaled']] = (60 + (data_df['A2'] - 60) / 4) / 75
    # атаки в минуту
    data_df['A1perMIN'] = data_df['A1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A1perMIN'] > 4, ['A1perMIN']] = np.float32(4.0)
    data_df['A2perMIN'] = data_df['A2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A2perMIN'] > 4, ['A2perMIN']] = np.float32(4.0)
    # динамика атак
    if relative:
        new_match_vector5 = data_df['Id'] != data_df['Id'].shift(5)
        data_df['A1relativ'] = data_df['A1'].astype(np.float32) - data_df['A1'].shift(5).astype(np.float32)
        data_df.loc[new_match_vector5, ['A1relativ']] = np.float32(0.0)
        data_df['A1relativ'] = data_df['A1relativ'].fillna(0)
        data_df.loc[data_df['A1relativ'] > 15, ['A1relativ']] = np.float32(15.)
        data_df['A2relativ'] = data_df['A2'].astype(np.float32) - data_df['A2'].shift(5).astype(np.float32)
        data_df.loc[new_match_vector5, ['A2relativ']] =  np.float32(0.0)
        data_df['A2relativ'] = data_df['A2relativ'].fillna(0)
        data_df.loc[data_df['A2relativ'] > 15, ['A2relativ']] = np.float32(15.)
    if 'A1' not in remain:
        data_df = data_df.drop(['A1', 'A2'], axis = 1)
    print('3. Атаки посчитаны...')
    # трансформируем опасные атаки
    data_df['DA1_scaled'] = data_df['DA1'].astype(np.float32) / 50
    data_df.loc[data_df['DA1'] >= 40, ['DA1_scaled']] = (80 + (data_df['DA1'] - 40) / 3) / 100
    data_df['DA2_scaled'] = data_df['DA2'].astype(np.float32) / 50
    data_df.loc[data_df['DA2'] >= 40, ['DA2_scaled']] = (80 + (data_df['DA2'] - 40) / 3) / 100
    # опасные атаки в минуту    
    data_df['DA1perMIN'] = data_df['DA1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA1perMIN'] > 3, ['DA1perMIN']] = np.float32(3.0)
    data_df['DA2perMIN'] = data_df['DA2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA2perMIN'] > 3, ['DA2perMIN']] = np.float32(3.0)
    # динамика опасных атак
    if relative:
        data_df['DA1relativ'] = data_df['DA1'].astype(np.float32) - data_df['DA1'].shift(5).astype(np.float32)
        data_df.loc[new_match_vector5, ['DA1relativ']] = np.float32(0.0)
        data_df['DA1relativ'] = data_df['DA1relativ'].fillna(0)
        data_df.loc[data_df['DA1relativ'] > 10, ['DA1relativ']] = np.float32(10.)
        data_df['DA2relativ'] = data_df['DA2'].astype(np.float32) - data_df['DA2'].shift(5).astype(np.float32)
        data_df.loc[new_match_vector5, ['DA2relativ']] = np.float32(0.0)
        data_df['DA2relativ'] = data_df['DA2relativ'].fillna(0)
        data_df.loc[data_df['DA2relativ'] > 10, ['DA2relativ']] = np.float32(10.)
    if 'DA1' not in remain:
        data_df = data_df.drop(['DA1', 'DA2'], axis = 1)
    if 'Minute' not in remain:
        data_df = data_df.drop(['Minute'], axis = 1)
    print('4. Опасные атаки посчитаны...')
    # трансформируем владение мячом
    data_df[data_df['Pos1'].isna() & new_match_vector] = 0
    data_df['Pos1_cleaned'] = data_df['Pos1'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos1_cleaned'] < 0.2, ['Pos1_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos1_cleaned'] > 0.8, ['Pos1_cleaned']] = np.float32(0.8)
    data_df[data_df['Pos2'].isna() & new_match_vector] = 0
    data_df['Pos2_cleaned'] = data_df['Pos2'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos2_cleaned'] < 0.2, ['Pos2_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos2_cleaned'] > 0.8, ['Pos2_cleaned']] = np.float32(0.8)
    if 'Pos1' not in remain:
        data_df = data_df.drop(['Pos1', 'Pos2'], axis = 1)
    print('5. Владение мячом посчитпно...')
    # трансформируем удары
    data_df[data_df['Off1'].isna() & new_match_vector] = 0
    data_df['Off1_norm'] = data_df['Off1'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off1_norm'] > 1.0, ['Off1_norm']] = np.float32(1.0)
    data_df[data_df['Off2'].isna() & new_match_vector] = 0
    data_df['Off2_norm'] = data_df['Off2'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off2_norm'] > 1.0, ['Off2_norm']] = np.float32(1.0)
    if 'Off1' not in remain:
        data_df = data_df.drop(['Off1', 'Off2'], axis = 1)
    print('6. Удары посчитаны...')
    # трансформируем удары в створ
    data_df[data_df['On1'].isna() & new_match_vector] = 0
    data_df['On1_norm'] = data_df['On1'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On1_norm'] > 1.0, ['On1_norm']] = np.float32(1.0)
    data_df[data_df['On2'].isna() & new_match_vector] = 0
    data_df['On2_norm'] = data_df['On2'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On2_norm'] > 1.0, ['On2_norm']] = np.float32(1.0)
    if 'On1' not in remain:
        data_df = data_df.drop(['On1', 'On2'], axis = 1)    
    print('7. Удары в створ посчитаны...')
    # трансформируем желтые карточки
    data_df[data_df['YC1'].isna() & new_match_vector] = 0
    data_df['YC1_transformed'] = data_df['YC1'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC1_transformed'] > 1.0, ['YC1_transformed']] = np.float32(1.0)
    data_df[data_df['YC2'].isna() & new_match_vector] = 0
    data_df['YC2_transformed'] = data_df['YC2'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC2_transformed'] > 1.0, ['YC2_transformed']] = np.float32(1.0)
    if 'YC1' not in remain:
        data_df = data_df.drop(['YC1', 'YC2'], axis = 1)
    print('8. Жёлтые карточки посчитаны...')
    # трансформируем красные карточки
    data_df[data_df['RC1'].isna() & new_match_vector] = 0
    data_df['RC1_transformed'] = data_df['RC1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC1_transformed'] > 1, ['RC1_transformed']] = np.int8(1)
    data_df[data_df['RC2'].isna() & new_match_vector] = 0
    data_df['RC2_transformed'] = data_df['RC2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC2_transformed'] > 1, ['RC2_transformed']] = np.int8(1)
    if 'RC1' not in remain:
        data_df = data_df.drop(['RC1', 'RC2'], axis = 1)
    print('9. Красные карточки посчитаны...')
    # трансформируем замены
    data_df[data_df['Sub1'].isna() & new_match_vector] = 0
    data_df['Sub1_transformed'] = data_df['Sub1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub1_transformed'] > 1, ['Sub1_transformed']] = np.int8(1)
    data_df[data_df['Sub2'].isna() & new_match_vector] = 0
    data_df['Sub2_transformed'] = data_df['Sub2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub2_transformed'] > 1, ['Sub2_transformed']] = np.int8(1)
    if 'Sub1' not in remain:
        data_df = data_df.drop(['Sub1', 'Sub2'], axis = 1)
    print('10. Замены посчитаны...')
    # трансформируем угловык
    data_df[data_df['Cor1'].isna() & new_match_vector] = 0
    data_df['Cor1_transformed'] = data_df['Cor1'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor1_transformed'] > 1.0, ['Cor1_transformed']] = np.float32(1.0)
    data_df[data_df['Cor2'].isna() & new_match_vector] = 0
    data_df['Cor2_transformed'] = data_df['Cor2'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor2_transformed'] > 1.0, ['Cor2_transformed']] = np.float32(1.0)
    if 'Cor1' not in remain:
        data_df = data_df.drop(['Cor1', 'Cor2'], axis = 1)
    print('11. Угловые посчитаны...')
    # трансформируем линию
    data_df['P1_transformed'] = np.log(data_df['P1'], dtype = np.float32) / 2
    data_df['P2_transformed'] = np.log(data_df['P2'], dtype = np.float32) / 2
    if 'P1' not in remain:
        data_df = data_df.drop(['P1', 'P2'], axis = 1)
    print('12. Линии посчитаны...')
    if 'Pen1' not in remain:
        data_df = data_df.drop(['Pen1', 'Pen2'], axis = 1)
    if 'Active' not in remain:
        data_df = data_df.drop(['Active'], axis = 1)
    return data_df

#### Predefine data

In [13]:
import datetime
data_types_dict = {
    'Id':np.int32, 
    'StatTime':str, 
    'Minute':np.int8, 
    'Active': np.int8, 'Score1':np.int8, 'Score2':np.int8,
    'A1':np.int16, 'A2':np.int16, 'DA1':np.int16, 'DA2':np.int16, 'Pos1':np.float32, 'Pos2':np.float32,
    'Off1':np.int8, 'Off2':np.int8, 'On1':np.int8, 'On2':np.int8, 'YC1':np.int8, 'YC2':np.int8,
    'RC1':np.int8, 'RC2':np.int8, 'Sub1':np.int8, 'Sub2':np.int8, 'Pen1':np.int8, 'Pen2':np.int8,
    'Cor1':np.int8, 'Cor2':np.int8, 'Period':np.int8, 
    'D':np.datetime64,
    'I':np.int32, 'Active.1':np.int8, 
    'Time':np.datetime64, 
    'Minute.1':np.int8, 
    'RawTime':np.datetime64, 
    'Score1.1':np.int8, 'Score2.1':np.int8, 'Period.1':np.int8, 
    'W1':np.float16, 'WX':np.float16, 'W2':np.float16, 'X1':np.float16, 'X2':np.float16, 'W12':np.float16, 'TotalValue':np.float16, 
    'Over':np.float16, 'Under':np.float16, 'Hand1Value':np.float16, 'H1':np.float16, 'H2':np.float16 
}

In [14]:
k_cols = [
    'W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'TotalValue' ,'Over',
    'Under', 'Hand1Value', 'H1', 'H2' 
]

In [15]:
drop_cols = [
    #'StatTime', 
    'Comment', 'D', 'I', 'Time', 'Minute.1', 'RawTime',
    'Score1.1', 'Score2.1', 'Period.1', 'Period', 'Periods', 'Serve'
]

#### Load Parameters

In [16]:
info_cols = ['Id', 'Result1', 'Result2']
prem_cols = ['Id', 'P1', 'PX', 'P2']
res_cols = [cols for cols in data_types_dict if cols not in drop_cols]

info_folder = './Index2/'
prem_folder = './Prem2/'
data_folder = './Result2/'

In [17]:
### собрать датасет с даты:
year_start = '2023'
month_start = '01'
day_start = '01'
### до даты
year_end = ''
month_end = ''
day_end = ''
date_start = year_start + month_start + day_start
date_end = year_end + month_end + day_end

#### Create DataFrame

In [18]:
info_dict = create_id_dict(date_start, info_folder, info_cols)
prem_dict = create_id_dict(date_start, prem_folder, prem_cols)
if len(prem_dict) != len(info_dict):
    print('Нет в info: ', set(prem_dict) - set(info_dict))
    print('Нет в prem: ', set(info_dict) - set(prem_dict))

In [19]:
files_list =['./Result2/' + str(id) + '.csv' 
    for id in set(info_dict)
    if os.path.exists('./Result2/' + str(id) + '.csv')]

In [26]:
time_point1 = time.time()
data_df = pd.concat(map(functools.partial(
    pd.read_csv, 
    sep = ';',
    #parse_dates = ['StatTime'],
    usecols = res_cols
                                    ), files_list), ignore_index=True)
time_point2 = time.time()
print(f'create dataframe: {time_point2 - time_point1} sec.')
#---------------
process = psutil.Process(os.getpid())
print(round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
#---------------  

data_df[k_cols] = data_df[k_cols].fillna(0).values # Заполняем пропущенные лайв кэфы 1
new_match_vector = data_df['Id'] != data_df['Id'].shift(1) # Отмечаем переходы между матчами в колонуах
data_cols = list(set(res_cols) - set(k_cols)) # Отбирвем колонки для заполнения пропусков метдом ffill
for col in data_cols:
    data_df.loc[data_df[col].isna() & new_match_vector, col] = 0 # Сами переходы с пропусками заполняем 0
data_df = data_df.fillna(method = 'ffill')
################ Записываем файл с временными метками и сбрасываем время матчей
#(data_df['Id'].astype(str) + ':' + data_df['StatTime'].astype(str).str.split(
#        ' ', expand=True)[0]
#        ).drop_duplicates().str.split(':', expand = True).rename(
#                                                columns = {0:'Id', 1:'StatTime'}
#                                                            ).to_csv('./time.csv', index = False)
#data_df = data_df.drop(['StatTime'], axis = 1)
############################################### Присоединяем результаты и кэфы
data_df[['Result1', 'Result2']] = [
    [info_dict[id]['Result1'], info_dict[id]['Result2']] 
    for id in tqdm(data_df['Id'].values, total = len(data_df))
    ]
data_df[['P1', 
         #'PX', 
         'P2']] = [
    [
        prem_dict[id]['P1'], 
        #prem_dict[id]['PX'], 
        prem_dict[id]['P2']] 
    for id in tqdm(data_df['Id'].values, total = len(data_df))
    ]

print('\n')
#---------------
process = psutil.Process(os.getpid())
print(round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
#---------------  

create dataframe: 125.07413363456726 sec.
2.34 GiB


100%|██████████| 1224315/1224315 [00:02<00:00, 579360.59it/s]




2.37 GiB


In [27]:
data_df.head()

,Id,StatTime,Minute,Active,Score1,Score2,A1,A2,DA1,DA2,Pos1,Pos2,Off1,Off2,On1,On2,YC1,YC2,RC1,RC2,Sub1,Sub2,Pen1,Pen2,Cor1,Cor2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,P1,P2
0,6291479,18.02.2023 12:50:44,1,1,0,0,1,1,0,1,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,2.05,3.70,3.06,1.33,1.680,1.23,3.0,1.90,1.88,0.0,1.58,2.33,0,1,2.0,3.1
1,6291479,18.02.2023 12:52:04,2,1,0,0,3,2,0,1,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,2.06,3.68,3.05,1.33,1.670,1.24,3.0,1.93,1.85,0.0,1.58,2.33,0,1,2.0,3.1
2,6291479,18.02.2023 12:53:25,4,1,0,0,4,2,1,1,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,2.07,3.60,3.08,1.32,1.666,1.24,3.0,2.07,1.74,0.0,1.58,2.33,0,1,2.0,3.1
3,6291479,18.02.2023 12:54:58,5,1,0,0,5,3,1,2,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,2.08,3.58,3.08,1.32,1.660,1.25,3.0,2.13,1.70,0.0,1.58,2.33,0,1,2.0,3.1
4,6291479,18.02.2023 12:55:58,6,1,0,0,5,5,1,4,0.0,0.0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,2.08,3.55,3.10,1.32,1.660,1.25,2.5,1.70,2.13,0.0,1.59,2.31,0,1,2.0,3.1


#### create time.csv

In [28]:
(data_df['Id'].astype(str) + ':' + data_df['StatTime'].astype(str).str.split(
        ' ', expand=True)[0]
        ).drop_duplicates().str.split(':', expand = True).rename(
                                                columns = {0:'Id', 1:'StatTime'}
                                                            ).to_csv('./time_upd.csv', index = False)
data_df = data_df.drop(['StatTime'], axis = 1)

In [29]:
len(data_df)

1224315

#### transform dataset

In [30]:
data_df.loc[new_match_vector & (data_df['P1'].isna()), ['P1']] = data_df['W1'][new_match_vector & (data_df['P1'].isna())]
data_df.loc[new_match_vector & (data_df['P2'].isna()), ['P2']] = data_df['W2'][new_match_vector & (data_df['P2'].isna())]
data_df['P1'] = data_df['P1'].fillna(method = 'ffill').values
data_df['P2'] = data_df['P2'].fillna(method = 'ffill').values

In [31]:
data_df = transform_dataset(
    data_df, 
    remain = ['Score1', 'Score2'],
    relative = False
    )

1. Минуты посчитаны...
2. Голы посчитаны...
3. Атаки посчитаны...
4. Опасные атаки посчитаны...
5. Владение мячом посчитпно...
6. Удары посчитаны...
7. Удары в створ посчитаны...
8. Жёлтые карточки посчитаны...
9. Красные карточки посчитаны...
10. Замены посчитаны...
11. Угловые посчитаны...


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


12. Линии посчитаны...


In [32]:
# Из-за того что некоторые кэфы были пропущены, и заменены на нули в препроцессинге на их месте появились inf, удаляем
print('P1 -inf & P2 -inf: ', (data_df['P1_transformed'] == -np.inf).sum(), (data_df['P2_transformed'] == -np.inf).sum())
data_df = data_df[~(data_df['P1_transformed'] == -np.inf)]
data_df = data_df[~(data_df['P2_transformed'] == -np.inf)]

print('P1 NaN & P2 NaN: ', data_df['P1_transformed'].isna().sum(), data_df['P2_transformed'].isna().sum())

data_df = data_df[~data_df['P1_transformed'].isna()]
data_df = data_df[~data_df['P2_transformed'].isna()]

print('Score-Result error 1&2: ', (~(data_df['Result1'] - data_df['Score1']) >= 0).sum(), (~(data_df['Result1'] - data_df['Score1']) >= 0).sum())
data_df = data_df.loc[(data_df['Result1'] - data_df['Score1']) >= 0]
data_df = data_df.loc[(data_df['Result2'] - data_df['Score2']) >= 0]


#---------------
process = psutil.Process(os.getpid())
print('mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
#---------------    

P1 -inf & P2 -inf:  1054 1054
P1 NaN & P2 NaN:  0 0
Score-Result error 1&2:  20 20
mem usage:  3.04 GiB


In [33]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1223240 entries, 0 to 1224314
Data columns (total 55 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Id                1223240 non-null  int64  
 1   Score1            1223240 non-null  int64  
 2   Score2            1223240 non-null  int64  
 3   Active.1          1223240 non-null  int64  
 4   W1                1223240 non-null  float64
 5   WX                1223240 non-null  float64
 6   W2                1223240 non-null  float64
 7   X1                1223240 non-null  float64
 8   X2                1223240 non-null  float64
 9   W12               1223240 non-null  float64
 10  TotalValue        1223240 non-null  float64
 11  Over              1223240 non-null  float64
 12  Under             1223240 non-null  float64
 13  Hand1Value        1223240 non-null  float64
 14  H1                1223240 non-null  float64
 15  H2                1223240 non-null  float64
 16  

In [34]:
data_df.describe()

,Id,Score1,Score2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,min_norm,Score1_norm,Score2_norm,Score_diff,Score_cat_1,Score_cat_2,Score_cat_3,Score_cat_4,Score_cat_5,Score_cat_6,Score_cat_7,Score_cat_8,Score_cat_9,A1_scaled,A2_scaled,A1perMIN,A2perMIN,DA1_scaled,DA2_scaled,DA1perMIN,DA2perMIN,Pos1_cleaned,Pos2_cleaned,Off1_norm,Off2_norm,On1_norm,On2_norm,YC1_transformed,YC2_transformed,RC1_transformed,RC2_transformed,Sub1_transformed,Sub2_transformed,Cor1_transformed,Cor2_transformed,P1_transformed,P2_transformed
count,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06,1.223240e+06
mean,5.985329e+06,2.983863e-01,2.400453e-01,8.742266e-01,3.043663e+00,3.608408e+00,4.613470e+00,1.329120e+00,1.769868e+00,1.057759e+00,2.353216e+00,1.706996e+00,1.684816e+00,-2.539914e-01,1.738253e+00,1.744618e+00,1.532956e+00,1.225130e+00,4.584769e-01,7.446658e-02,5.996881e-02,1.450574e-02,6.605409e-04,3.552860e-03,1.980887e-02,1.252354e-01,6.597928e-01,1.545191e-01,2.904908e-02,5.965305e-03,1.415912e-03,3.275578e-01,3.114248e-01,1.106338e+00,1.051965e+00,2.689399e-01,2.417156e-01,5.842383e-01,5.246256e-01,4.294243e-01,4.055518e-01,1.430056e-01,1.225209e-01,1.977681e-01,1.674516e-01,1.118444e-01,1.244981e-01,6.077303e-03,6.971649e-03,2.663337e-02,2.827900e-02,1.890749e-01,1.605529e-01,4.223377e-01,5.986543e-01
std,4.263312e+05,5.944298e-01,5.271992e-01,3.315939e-01,4.178030e+00,2.543143e+00,5.886139e+00,1.284099e+00,1.670416e+00,4.787107e-01,1.253852e+00,6.223470e-01,6.107300e-01,1.185612e+00,7.098662e-01,7.122908e-01,1.402802e+00,1.248968e+00,2.593684e-01,1.476283e-01,1.314469e-01,1.921436e-01,2.569251e-02,5.949991e-02,1.393431e-01,3.309858e-01,4.737789e-01,3.614458e-01,1.679442e-01,7.700471e-02,3.760197e-02,2.066836e-01,1.981929e-01,4.363853e-01,4.246509e-01,1.995743e-01,1.850384e-01,3.236830e-01,3.085980e-01,1.767260e-01,1.652045e-01,1.633827e-01,1.464586e-01,2.412493e-01,2.208958e-01,2.444014e-01,2.560772e-01,7.771984e-02,8.320487e-02,1.610095e-01,1.657689e-01,2.247133e-01,2.052770e-01,2.548604e-01,2.976166e-01
min,4.975839e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-02,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-01,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.989885e-04,2.493768e-03
25%,5.539179e+06,0.000000e+00,0.000000e+00,1.000000e+00,1.300000e+00,2.780000e+00,1.570000e+00,1.022000e+00,1.104000e+00,1.098000e+00,1.500000e+00,1.740000e+00,1.720000e+00,-1.000000e+00,1.580000e+00,1.590000e+00,1.000000e+00,0.000000e+00,2.400000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.600000e-01,1.466667e-01,8.333333e-01,7.894737e-01,1.000000e-01,1.000000e-01,3.684210e-01,3.225806e-01,2.000000e-01,2.000000e-01,0.000000e+00,0.000000

In [35]:
data_df.columns

Index(['Id', 'Score1', 'Score2', 'Active.1', 'W1', 'WX', 'W2', 'X1', 'X2',
       'W12', 'TotalValue', 'Over', 'Under', 'Hand1Value', 'H1', 'H2',
       'Result1', 'Result2', 'min_norm', 'Score1_norm', 'Score2_norm',
       'Score_diff', 'Score_cat_1', 'Score_cat_2', 'Score_cat_3',
       'Score_cat_4', 'Score_cat_5', 'Score_cat_6', 'Score_cat_7',
       'Score_cat_8', 'Score_cat_9', 'A1_scaled', 'A2_scaled', 'A1perMIN',
       'A2perMIN', 'DA1_scaled', 'DA2_scaled', 'DA1perMIN', 'DA2perMIN',
       'Pos1_cleaned', 'Pos2_cleaned', 'Off1_norm', 'Off2_norm', 'On1_norm',
       'On2_norm', 'YC1_transformed', 'YC2_transformed', 'RC1_transformed',
       'RC2_transformed', 'Sub1_transformed', 'Sub2_transformed',
       'Cor1_transformed', 'Cor2_transformed', 'P1_transformed',
       'P2_transformed'],
      dtype='object')

In [36]:
data_df

,Id,Score1,Score2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,min_norm,Score1_norm,Score2_norm,Score_diff,Score_cat_1,Score_cat_2,Score_cat_3,Score_cat_4,Score_cat_5,Score_cat_6,Score_cat_7,Score_cat_8,Score_cat_9,A1_scaled,A2_scaled,A1perMIN,A2perMIN,DA1_scaled,DA2_scaled,DA1perMIN,DA2perMIN,Pos1_cleaned,Pos2_cleaned,Off1_norm,Off2_norm,On1_norm,On2_norm,YC1_transformed,YC2_transformed,RC1_transformed,RC2_transformed,Sub1_transformed,Sub2_transformed,Cor1_transformed,Cor2_transformed,P1_transformed,P2_transformed
0,6291479,0,0,1,2.050,3.70,3.06,1.33,1.680,1.230,3.0,1.900,1.88,0.0,1.580,2.33,0,1,0.02,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.013333,0.013333,1.000000,1.000000,0.00,0.02,0.000000,1.000000,0.20,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.000000,0.000000,0.346574,0.565701
1,6291479,0,0,1,2.060,3.68,3.05,1.33,1.670,1.240,3.0,1.930,1.85,0.0,1.580,2.33,0,1,0.04,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.040000,0.026667,1.500000,1.000000,0.00,0.02,0.000000,0.500000,0.20,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.000000,0.000000,0.346574,0.565701
2,6291479,0,0,1,2.070,3.60,3.08,1.32,1.666,1.240,3.0,2.070,1.74,0.0,1.580,2.33,0,1,0.08,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.053333,0.026667,1.000000,0.500000,0.02,0.02,0.250000,0.250000,0.20,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.000000,0.000000,0.346574,0.565701
3,6291479,0,0,1,2.080,3.58,3.08,1.32,1.660,1.250,3.0,2.130,1.70,0.0,1.580,2.33,0,1,0.10,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.066667,0.040000,1.000000,0.600000,0.02,0.04,0.200000,0.400000,0.20,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.000000,0.000000,0.346574,0.565701
4,6291479,0,0,1,2.080,3.55,3.10,1.32,1.660,1.250,2.5,1.700,2.13,0.0,1.590,2.31,0,1,0.12,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.066667,0.066667,0.833333,0.833333,0.02,0.08,0.166667,0.666667,0.20,0.20,0.0,0.0,0.0,0.2,0.0,0.0,0,0,0,0,0.000000,0.166667,0.346574,0.565701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224310,6160370,1,0,1,1.052,9.20,29.00,0.00,7.100,1.017,2.5,1.880,1.88,-2.0,2.130,1.68,1,0,0.82,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.746667,0.560000,1.365854,1.024390,0.52,0.36,0.634146,0.439024,0.58,0.42,0.3,0.2,1.0,0.6,0.0,0.0,0,0,0,0,0.166667,0.000000,0.154844,1.039721
1224311,6160370,1,0,1,1.053,9.10,29.00,0.00,7.000,1.018,2.5,1.940,1.82,-2.0,2.200,1.64,1,0,0.84,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.786667,0.573333,1.404762,1.023810,0.54,0.36,0.642857,0.428571,0.59,0.41,0.3,0.2,1.0,0.6,0.0,0.0,0,0,0,0,0.166667,0.000000,0.154844,1.039721
1224312,6160370,1,0,1,1.053,9.10,29.00,0.00,7.000,1.018,2.5,1.952,1.81,-2.0,2.240,1.62,1,0,0.86,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.786667,0.613333,1.372093,1.069767,0.54,0.42,0.627907,0.488372,0.57,0.43,0.3,0.2,1.0,0.6,0.0,0.0,0,0,0,0,0.166667,0.000000,0.154844,1.039721
1224313,6160370,1,0,1,1.053,9.10,29.00,0.00,7.000,1.018,2.5,1.990,1.78,-1.5,1.615,2.25,1,0,0.88,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.786667,0.613333,1.340909,1.045455,0.54,0.44,0.613636,0.500000,0.57,0.43,0.3,0.2,1.0,0.6,0.0,0.0,0,0,0,0,0.166667,0.000000,0.154844,1.039721


#### split data & prepare files for upload

In [44]:
#%%writefile save_discription_upd.txt
np.savez_compressed('./dataset_upd', 
                    X = data_df[data_df.columns[18:]].values,
                    y_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8),
                    y_multi = 1 - np.sign((data_df['Result1'] - data_df['Result2']).values),
                    y_diff = (data_df['Result1'] - data_df['Result2']).values,
                    y_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values,
                    y_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values,
                    score1 = data_df['Score1'].values,
                    score2 = data_df['Score2'].values,
                    result1 = data_df['Result1'].values,
                    result2 = data_df['Result2'].values,
                    K_train = data_df[data_df.columns[3:16]].values,
                )

Writing save_discription_upd.txt


In [43]:
np.savez_compressed('./additional_data_upd', 
                    id = data_df['Id'].values,
                    min = data_df['min_norm'].values,
)

In [42]:
params = {}
params['description'] = 'холдаут часть https://disk.yandex.ru/d/w5i8F_MEgrPOGQ \n' + \
'4 класса задач, бинарная классификация и мультикласс, и под регрессию 2 варианта без разделения на трейн, тест и холдаут'
params['features'] = list(data_df.columns)

#### upload data to neptune.ai

In [45]:
data_version = 'football_live_upd_230510/'
project = neptune.init_project(
    project="scomesse/football", 
    api_token = api_key
    )
project[data_version + 'dataset'].upload('./dataset_upd.npz')
project[data_version + 'description'].upload('./save_discription_upd.txt')
project[data_version + 'additional_data'].upload('./additional_data_upd.npz')
project[data_version + 'info'].upload('/content/Index2/info.csv')
project[data_version + 'time'].upload('./time_upd.csv')
project[data_version + 'params'] = stringify_unsupported(params)
project.stop()

https://app.neptune.ai/scomesse/football/
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 7 operations to synchronize with Neptune. Do not kill this process.


ERROR:neptune.internal.operation_processors.async_operation_processor:Error occurred during asynchronous operation processing: Cannot upload file /content/Index2/info.csv: Path not found or is a not a file.


All 7 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/metadata
